In [37]:
import os
import scipy
import numpy as np
import trimesh
from trimesh import viewer
from trimesh.transformations import translation_matrix, rotation_matrix, concatenate_matrices
from scipy.spatial.transform import Rotation
from scipy.spatial import cKDTree

np.set_printoptions(precision=3, suppress=True, linewidth=np.inf)


In [38]:
mesh1 = trimesh.exchange.load.load('wolf1.ply')
mesh2 = mesh1.copy()

# apply an arbitrary transformation to mesh1
mesh1 = mesh1.slice_plane(plane_normal=[0, -1, 0], plane_origin=[10, 10, 10])
T = translation_matrix([0, 0, 0])
xaxis, yaxis, zaxis = [1, 0, 0], [0, 1, 0], [0, 0, 1]
Rx = rotation_matrix(0, xaxis)
Ry = rotation_matrix(np.pi/8, yaxis)
Rz = rotation_matrix(0, zaxis)
M = concatenate_matrices(T, Rx, Ry, Rz)
mesh1.apply_transform(M)

# write scene to html
if os.path.exists("scene.html"):
    os.remove("scene.html")
html = viewer.notebook.scene_to_html((trimesh.util.concatenate([mesh1, mesh2])).scene())
with open("scene.html", "w") as file:
    file.write(html)


In [39]:
def register(source, target, w1=0.1, w2=0.1, w3=1.0, max_iter=100):
    # resulting transformation is (R, t)
    transform = (np.eye(3), np.zeros(3))
    x = source.vertices
    y = target.vertices
    yn = target.vertex_normals
    print(yn)
    N = len(x)
    z = x.copy()
    tree = cKDTree(y)
    for iter in range(max_iter):
        _, i = tree.query(z, k=1)
        # applies transformation to original x
        x_t = x.dot(transform[0]) + transform[1]
        # build a matrix equivalent to cross product
        X_t = np.stack([
            np.stack((np.zeros(len(x_t)), -x_t[:, 2], x_t[:, 1]), axis=1),
            np.stack((x_t[:, 2], np.zeros(len(x_t)), -x_t[:, 0]), axis=1),
            np.stack((-x_t[:, 1], x_t[:, 0], np.zeros(len(x_t))), axis=1)
        ], axis=1)
        # build blocks of sparse coefficient matrix A
        A_rr = np.einsum('abc,adc->bd', X_t, X_t)
        A_tr = -np.sum(X_t, axis=0)
        A_zir = np.concatenate([X_ti for X_ti in X_t], axis=1)
        A_rt = A_tr.T
        A_tt = N * np.eye(3)
        A_zit = -np.tile(np.eye(3), len(x_t))
        A_rzi = A_zir.T
        A_tzi = A_zit.T
        A_zizj = scipy.sparse.eye(3 * N) + \
                 w1 / w3 * scipy.sparse.eye(3 * N) + \
                 w2 / w3 * scipy.sparse.block_diag(np.einsum('ab,ac->abc', yn[i], yn[i]), format='csr')
        A_rr = scipy.sparse.csr_matrix(A_rr)
        A_tr = scipy.sparse.csr_matrix(A_tr)
        A_zir = scipy.sparse.csr_matrix(A_zir)
        A_rt = scipy.sparse.csr_matrix(A_rt)
        A_tt = scipy.sparse.csr_matrix(A_tt)
        A_zit = scipy.sparse.csr_matrix(A_zit)
        A_rzi = scipy.sparse.csr_matrix(A_rzi)
        A_tzi = scipy.sparse.csr_matrix(A_tzi)
        A_zizj = scipy.sparse.csr_matrix(A_zizj)
        # assemble coefficient matrix A
        A = scipy.sparse.bmat([[A_rr, A_tr, A_zir],
                               [A_rt, A_tt, A_zit],
                               [A_rzi, A_tzi, A_zizj]], format='csr')
        # build blocks of result vector b
        b_r = np.zeros(3)
        b_t = -np.sum(x_t, axis=0)
        b_zi = (w1 / w3 * y[i] + w2 / w3 * np.einsum('ab,ac,ac->ab', yn[i], yn[i], y[i]) + x_t).flatten()
        # assemble result vector b
        b = np.concatenate([b_r, b_t, b_zi])
        # solve system of linear equations Ax = b using conjugate gradient method
        solution, info = scipy.sparse.linalg.cg(A, b, tol=1e-10, maxiter=1000)
        # ensure convergence
        assert info == 0
        # update resulting transformation
        r = solution[:3]
        t = solution[3:6]
        z = solution[6:].reshape(-1, 3)
        R = Rotation.from_euler('xyz', r).as_matrix()
        # ensure that R is a rotation matrix
        assert np.isclose(np.linalg.det(R), 1.0)
        transform = (np.dot(transform[0], R), transform[1] + t)
        print(iter, '\n', transform, '\n')
    return transform, z


In [40]:
(R, t), z = register(mesh1, mesh2, w1=0, max_iter=100)
print('ground truth\n', M[:3, :3], M[:3, 3])
trans_mesh = trimesh.base.Trimesh(mesh1.vertices.dot(R) + t, mesh1.faces) # z is also interesting

if os.path.exists("scene.html"):
    os.remove("scene.html")
html = viewer.notebook.scene_to_html(trimesh.util.concatenate([trans_mesh, mesh2]).scene())
with open("scene.html", "w") as file:
    file.write(html)


[[-0.428  0.76  -0.489]
 [-0.203  0.621 -0.757]
 [-0.029  0.723 -0.691]
 ...
 [-0.292  0.809  0.51 ]
 [-0.555  0.787  0.27 ]
 [-0.606  0.782 -0.146]]
0 
 (array([[ 0.974, -0.12 ,  0.194],
       [ 0.131,  0.99 , -0.047],
       [-0.187,  0.072,  0.98 ]]), array([ 0.892, -0.43 , -0.819])) 

1 
 (array([[ 0.961, -0.109,  0.256],
       [ 0.125,  0.991, -0.046],
       [-0.248,  0.076,  0.966]]), array([ 2.375, -1.133, -0.439])) 

2 
 (array([[ 0.955, -0.084,  0.284],
       [ 0.099,  0.994, -0.038],
       [-0.279,  0.065,  0.958]]), array([ 2.021, -1.339, -0.121])) 

3 
 (array([[ 0.95 , -0.054,  0.307],
       [ 0.067,  0.997, -0.033],
       [-0.305,  0.052,  0.951]]), array([ 1.484, -1.204, -0.228])) 

4 
 (array([[ 0.944, -0.038,  0.329],
       [ 0.047,  0.999, -0.02 ],
       [-0.328,  0.034,  0.944]]), array([ 1.305, -1.132,  0.096])) 

5 
 (array([[ 0.939, -0.033,  0.341],
       [ 0.04 ,  0.999, -0.015],
       [-0.341,  0.028,  0.94 ]]), array([ 1.416, -0.925,  0.131])) 

6 
 